In [1]:
import sys
import os
import pandas as pd
sys.path.append(os.path.abspath(os.path.join('..')))

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()  # Carga las variables del archivo .env

config = {
    'host': os.getenv('HOST'),
    'user': os.getenv('USER'),
    'password': os.getenv('PASSWORD'),
    'database': os.getenv('DATABASE'),
    'port': os.getenv('PORT')
}

In [3]:
import mysql.connector

connection = mysql.connector.connect(**config)


In [4]:
cursor = connection.cursor()


<center><h1><b>Generación Tablas</b></h1</center>

In [5]:
# Ejecuta DROP y luego la generación de la tabla para reproducibilidad
cursor.execute("DROP TABLE IF EXISTS caso_5.weekly_sales;")
cursor.execute("""
CREATE TABLE caso_5.weekly_sales (
    week_date VARCHAR(7),
    region VARCHAR(13),
    platform VARCHAR(7),
    segment VARCHAR(4),
    customer_type VARCHAR(8),
    transactions INT,
    sales INT
);
""")
connection.commit()

In [6]:
from utils.funciones_utiles import load_text_as_tuples, verifica_tabla

cursor.executemany(
    '''INSERT INTO caso_5.weekly_sales VALUES (%s, %s, %s, %s, %s, %s, %s)''', 
    load_text_as_tuples('datos_tablas/data_weekly_sales.txt'))

connection.commit()


In [7]:
verifica_tabla('weekly_sales', connection)

c:\Proyectos\SQL_DESAFIOS\SQL_DESAFIO\utils\funciones_utiles.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(f'''SELECT * FROM {table_name} LIMIT 5''', connection)


,week_date,region,platform,segment,customer_type,transactions,sales
0,31/8/20,ASIA,Retail,C3,New,120631,3656163
1,31/8/20,ASIA,Retail,F1,New,31574,996575
2,31/8/20,USA,Retail,null,Guest,529151,16509610
3,31/8/20,EUROPE,Retail,C1,New,4517,141942
4,31/8/20,AFRICA,Retail,C2,New,58046,1758388


---

## **REVISION**

## Revisando nulos

In [8]:
query = '''
SELECT 
    SUM(CASE WHEN week_date IS NULL OR LOWER(week_date) = 'null' THEN 1 ELSE 0 END) AS week_date_nulls,
    SUM(CASE WHEN region IS NULL OR LOWER(region) = 'null' THEN 1 ELSE 0 END) AS region_nulls,
    SUM(CASE WHEN platform IS NULL OR LOWER(platform) = 'null' THEN 1 ELSE 0 END) AS platform_nulls,
    SUM(CASE WHEN segment IS NULL OR LOWER(segment) = 'null' THEN 1 ELSE 0 END) AS segment_nulls,
    SUM(CASE WHEN customer_type IS NULL OR LOWER(customer_type) = 'null' THEN 1 ELSE 0 END) AS customer_type_nulls,
    SUM(CASE WHEN transactions IS NULL OR transactions = 'null' THEN 1 ELSE 0 END) AS transactions_nulls,
    SUM(CASE WHEN sales IS NULL OR sales = 'null' THEN 1 ELSE 0 END) AS sales_nulls
FROM caso_5.weekly_sales;


'''


pd.read_sql_query(query, connection)

C:\Users\kev_g\AppData\Local\Temp\ipykernel_11936\1639012929.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query, connection)


,week_date_nulls,region_nulls,platform_nulls,segment_nulls,customer_type_nulls,transactions_nulls,sales_nulls
0,0.0,0.0,0.0,3024.0,0.0,0.0,1.0


🎇 **Insight:**

Dentro de todas las columnas existen dos que poseen datos nulos, `segment` y `sales_nulls` los cuales consisten en NULL o null como str. Ahora dentro de todas estas columnas podrian existir mas datos invalidos los que deberán ser revisados.

In [9]:
query = '''
SELECT 
    column_name,
    data_type
FROM information_schema.columns
WHERE table_schema = 'caso_5'
    AND table_name = 'weekly_sales';

'''


pd.read_sql_query(query, connection)

C:\Users\kev_g\AppData\Local\Temp\ipykernel_11936\2602599584.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query, connection)


,COLUMN_NAME,DATA_TYPE
0,customer_type,varchar
1,platform,varchar
2,region,varchar
3,sales,int
4,segment,varchar
5,transactions,int
6,week_date,varchar


🎇 **Insight:**

Las columnas parecen estar bien a excepción de week_date que el tipo de dato es VARCHAR y deberia ser DATE.

In [10]:
query = '''
    SELECT DISTINCT 
        region 
    FROM weekly_sales
    UNION 

    SELECT '----platform_unicos----'

    UNION 

    SELECT DISTINCT 
        platform
    FROM weekly_sales
    UNION 

    SELECT '----segment_unicos----'

    UNION 

    SELECT DISTINCT 
        segment
    FROM weekly_sales

    UNION 

    SELECT '----customer_type_unicos----'

    UNION 

    SELECT DISTINCT 
        customer_type
    FROM weekly_sales;

'''


pd.read_sql_query(query, connection)

C:\Users\kev_g\AppData\Local\Temp\ipykernel_11936\3334628801.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query, connection)


,region
0,ASIA
1,USA
2,EUROPE
3,AFRICA
4,CANADA
5,OCEANIA
6,SOUTH AMERICA
7,----platform_unicos----
8,Retail
9,Shopify


🎇 **Insight:**

Las features categoricas sin problemas son `platform` y `customer_type`.

Por otro lado las features con problemas son 

`region`:

- Todos los registros en mayus
- Contiene registros de paises y deberian ser todos región ( ej USA/CANADA deberia ser AMERICA o NORTH AMERICA)


`segment_unicos`: 

- Contiene dato nullo representado por la string 'null'


In [11]:
query = '''
    SELECT 
        MIN(transactions) AS minimo_transaction,
        MAX(transactions) AS maximo_transaction, 
        MIN(sales) AS minimo_sales,
        MAX(sales) AS maximo_sales
    FROM
        weekly_sales
'''


pd.read_sql_query(query, connection)

C:\Users\kev_g\AppData\Local\Temp\ipykernel_11936\134195547.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query, connection)


,minimo_transaction,maximo_transaction,minimo_sales,maximo_sales
0,1,2578158,0,69763805


No parece existir algo que llame la atención o sea anomalo

# **1. Pasos para la limpieza de datos**

## Instrucciones SQL - Creación de tabla `clean_weekly_sales` en el esquema `caso_5`

En una sola consulta, realiza las siguientes operaciones y genera una nueva tabla llamada `clean_weekly_sales` dentro del esquema `caso_5`:

1. **Convertir `week_date` al formato `DATE`**  
   Asegúrate de que la columna `week_date` esté en formato fecha estándar.

2. **Agregar columna `week_number`**  
   Esta columna representará el número de la semana del año, donde:
   - Del 1 al 7 de enero → semana 1  
   - Del 8 al 14 de enero → semana 2  
   - etc.

3. **Agregar columna `month_number`**  
   Extrae el número del mes del campo `week_date` (por ejemplo: enero = 1, febrero = 2, etc.).

4. **Agregar columna `calendar_year`**  
   Extrae el año calendario (valores permitidos: 2018, 2019 o 2020) desde `week_date`.

5. **Agregar columna `age_band` después de la columna `segment`**  
   Según el número presente dentro del valor de `segment`, asigna:
   - `1` → "Young Adults"
   - `2` → "Middle Aged"
   - `3` o `4` → "Retirees"

6. **Agregar columna `demographic`**  
   Según la primera letra del valor en `segment`, asigna:
   - `C` → "Couples"
   - `F` → "Families"

7. **Manejar valores nulos**  
   Reemplaza cualquier valor `NULL` o cadena vacía en la columna original `segment`, así como en las nuevas columnas `age_band` y `demographic`, con el string `"unknown"`.

8. **Agregar columna `avg_transaction`**  
   Calcula el promedio de transacción como:  
   `sales / transactions`, redondeado a 2 decimales.


In [12]:
query = '''
    DROP TABLE IF EXISTS caso_5.clean_weekly_sales;

    CREATE TABLE caso_5.clean_weekly_sales AS 
    SELECT 
        STR_TO_DATE(week_date, '%d/%m/%Y') AS week_date,
        WEEK(STR_TO_DATE(week_date, '%d/%m/%Y')) AS week_number,
        MONTH(STR_TO_DATE(week_date, '%d/%m/%Y')) AS month_number,
        YEAR(STR_TO_DATE(week_date, '%d/%m/%Y')) AS calendar_year,
        region, 
        platform, 
        segment,
        CASE 
            WHEN RIGHT(segment,1) = '1' THEN 'Young Adults'
            WHEN RIGHT(segment,1) = '2' THEN 'Middle Aged'
            WHEN RIGHT(segment,1) IN ('3', '4') THEN 'Retirees' 
            ELSE 'unknown' END
            AS age_band,
        CASE 
            WHEN LEFT(segment,1) = 'C' THEN 'Couples'
            WHEN LEFT(segment,1) = 'F' THEN 'Families'
            ELSE 'unknown' END 
            AS demographic,
        
        transactions,
        ROUND(sales / NULLIF(transactions, 0), 2) AS avg_transaction,
        sales
        
       


    FROM
        weekly_sales 



'''


for result in cursor.execute(query, multi=True):
    if result.with_rows:
        result.fetchall() 

connection.commit()

<center><h1><b>Preguntas</b></h1</center>


# **1. ¿Qué día de la semana se utiliza para cada valor de `week_date`?**

In [13]:
query = '''
    SELECT 
        *
    FROM clean_weekly_sales

'''


pd.read_sql_query(query, connection)

C:\Users\kev_g\AppData\Local\Temp\ipykernel_11936\1654279346.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query, connection)


,week_date,week_number,month_number,calendar_year,region,platform,segment,age_band,demographic,transactions,avg_transaction,sales
0,2020-08-31,35,8,2020,ASIA,Retail,C3,Retirees,Couples,120631,30.31,3656163
1,2020-08-31,35,8,2020,ASIA,Retail,F1,Young Adults,Families,31574,31.56,996575
2,2020-08-31,35,8,2020,USA,Retail,null,unknown,unknown,529151,31.20,16509610
3,2020-08-31,35,8,2020,EUROPE,Retail,C1,Young Adults,Couples,4517,31.42,141942
4,2020-08-31,35,8,2020,AFRICA,Retail,C2,Middle Aged,Couples,58046,30.29,1758388
...,...,...,...,...,...,...,...,...,...,...,...,...
17112,2018-03-26,12,3,2018,AFRICA,Retail,C3,Retirees,Couples,98342,37.69,3706066
17113,2018-03-26,12,3,2018,USA,Shopify,C4,Retirees,Couples,16,174.00,2784
17114,2018-03-26,12,3,2018,USA,Retail,F2,Middle Aged,Families,25665,41.46,1064172
17115,2018-03-26,12,3,2018,EUROPE,Retail,C4,Retirees,Couples,883,37.96,33523


In [14]:
query = '''
SELECT 
    DISTINCT(DATE_FORMAT(week_date, '%W')) AS week_date_value
FROM clean_weekly_sales;

'''


pd.read_sql_query(query, connection)

C:\Users\kev_g\AppData\Local\Temp\ipykernel_11936\1753603731.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query, connection)


,week_date_value
0,Monday


# **2. ¿Qué rango de números de semana falta en el conjunto de datos?**

In [15]:
query = '''

-- Generar números del 1 al 52 con RECURSIVE
WITH RECURSIVE all_weeks AS (
    SELECT 1 AS week_number
    UNION ALL
    SELECT week_number + 1 FROM all_weeks WHERE week_number < 52
)


-- Detectar cuáles semanas están ausentes
SELECT 
    DISTINCT aw.week_number
FROM all_weeks aw
LEFT JOIN clean_weekly_sales c
ON aw.week_number = c.week_number
WHERE c.week_number IS NULL


'''


pd.read_sql_query(query, connection)

C:\Users\kev_g\AppData\Local\Temp\ipykernel_11936\1687227176.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query, connection)


,week_number
0,1
1,2
2,3
3,4
4,5
5,6
6,7
7,8
8,9
9,10


# **3. ¿Cuántas transacciones totales hubo para cada año en el conjunto de datos?**


In [16]:
query = '''

    SELECT 
        calendar_year, 
        SUM(transactions) AS total_transacciones
    FROM clean_weekly_sales
    GROUP BY calendar_year
    ORDER BY calendar_year;

'''


pd.read_sql_query(query, connection)

C:\Users\kev_g\AppData\Local\Temp\ipykernel_11936\2007027450.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query, connection)


,calendar_year,total_transacciones
0,2018,346406460.0
1,2019,365639285.0
2,2020,375813651.0


# **4. ¿Cuál es el total de ventas por región para cada mes?**

In [17]:
query = '''

    SELECT 
        region, 
        month_number,
        SUM(sales) AS total_amount
    FROM clean_weekly_sales
    GROUP BY month_number, region 
    ORDER BY month_number, region;

'''


pd.read_sql_query(query, connection)

C:\Users\kev_g\AppData\Local\Temp\ipykernel_11936\1775166071.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query, connection)


,region,month_number,total_amount
0,AFRICA,3,5.677675e+08
1,ASIA,3,5.297708e+08
2,CANADA,3,1.446343e+08
3,EUROPE,3,3.533709e+07
4,OCEANIA,3,7.832829e+08
5,SOUTH AMERICA,3,7.102311e+07
6,USA,3,2.253530e+08
7,AFRICA,4,1.911784e+09
8,ASIA,4,1.804629e+09
9,CANADA,4,4.845526e+08


# **5. ¿Cuál es el total de transacciones por cada plataforma?**

In [18]:
query = '''

    SELECT 
        platform,
        SUM(transactions) AS total_transactions
    FROM 
        clean_weekly_sales
    GROUP BY
        platform


'''


pd.read_sql_query(query, connection)

C:\Users\kev_g\AppData\Local\Temp\ipykernel_11936\930475379.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query, connection)


,platform,total_transactions
0,Retail,1.081934e+09
1,Shopify,5.925169e+06


# **6. ¿Cuál es el porcentaje de ventas de Retail vs Shopify para cada mes?**

In [19]:
query = '''
    
    -- Ventas por plataforma segun año y mes

    WITH platform_sales AS (
        SELECT
            calendar_year,
            month_number,
            platform,
            SUM(sales) AS total_sales
        FROM clean_weekly_sales
        GROUP BY calendar_year, month_number, platform
    ),

    -- Ventas totales mensuales

    monthly_sales AS (
        SELECT
            calendar_year,
            month_number,
            SUM(sales) AS total_sales_month
        FROM clean_weekly_sales
        GROUP BY calendar_year, month_number
    )

    -- join entre ambas tablas para dividir por monthly_Sales
    
    SELECT
        ps.calendar_year,
        ps.month_number,
        ps.platform,
        ps.total_sales,
        ms.total_sales_month,
        ROUND(100 * ps.total_sales / ms.total_sales_month, 2) AS porcentaje
    FROM platform_sales ps
    JOIN monthly_sales ms
        ON ps.calendar_year = ms.calendar_year
        AND ps.month_number = ms.month_number
    ORDER BY ps.calendar_year, ps.month_number, ps.platform;

'''


pd.read_sql_query(query, connection)

C:\Users\kev_g\AppData\Local\Temp\ipykernel_11936\2819239124.py:44: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query, connection)


,calendar_year,month_number,platform,total_sales,total_sales_month,porcentaje
0,2018,3,Retail,5.255831e+08,5.367555e+08,97.92
1,2018,3,Shopify,1.117239e+07,5.367555e+08,2.08
2,2018,4,Retail,2.617369e+09,2.672805e+09,97.93
3,2018,4,Shopify,5.543557e+07,2.672805e+09,2.07
4,2018,5,Retail,2.080290e+09,2.128656e+09,97.73
5,2018,5,Shopify,4.836594e+07,2.128656e+09,2.27
6,2018,6,Retail,2.061129e+09,2.108452e+09,97.76
7,2018,6,Shopify,4.732364e+07,2.108452e+09,2.24
8,2018,7,Retail,2.646368e+09,2.707198e+09,97.75
9,2018,7,Shopify,6.083018e+07,2.707198e+09,2.25


# **7. ¿Cuál es el porcentaje de ventas por grupo demográfico (`demographic`) para cada año en el conjunto de datos?**

In [20]:
query = '''
    
    -- Ventas segun demographic por año

    WITH yearly_demographic_sales AS (
        SELECT
            calendar_year,
            demographic,
            SUM(sales) AS total_sales
        FROM clean_weekly_sales
        GROUP BY calendar_year, demographic
    ),

    -- Ventas totales

    yearly_totals_sales AS (
        SELECT
            calendar_year,
            SUM(sales) AS total_sales_year
        FROM clean_weekly_sales
        GROUP BY calendar_year
    )

    -- Query final
    
    SELECT
        ys.calendar_year,
        ys.demographic,
        ys.total_sales,
        ROUND(100 * ys.total_sales / yt.total_sales_year, 2) AS porcentaje
    FROM 
        yearly_demographic_sales ys
    JOIN
        yearly_totals_sales yt
        ON ys.calendar_year = yt.calendar_year

'''


pd.read_sql_query(query, connection)

C:\Users\kev_g\AppData\Local\Temp\ipykernel_11936\1049706376.py:40: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query, connection)


,calendar_year,demographic,total_sales,porcentaje
0,2020,Couples,4.049567e+09,28.72
1,2020,Families,4.614338e+09,32.73
2,2020,unknown,5.436316e+09,38.55
3,2019,unknown,5.532862e+09,40.25
4,2019,Couples,3.749252e+09,27.28
5,2019,Families,4.463918e+09,32.47
6,2018,unknown,5.369434e+09,41.63
7,2018,Couples,3.402389e+09,26.38
8,2018,Families,4.125558e+09,31.99


# **8. ¿Qué combinaciones de `age_band` y `demographic` contribuyen más a las ventas de Retail?**


In [21]:
query = '''
    
    
WITH ventas AS (
    SELECT
        age_band,
        demographic,
        SUM(sales) AS total_sales     
    FROM clean_weekly_sales
    WHERE platform = 'Retail'
    GROUP BY age_band, demographic
)

SELECT
    age_band,
    demographic,
    total_sales,
    total_sales /
        SUM(total_sales) OVER ()     
        AS sales_share               
FROM ventas
ORDER BY total_sales DESC;
        

    

'''


pd.read_sql_query(query, connection)

C:\Users\kev_g\AppData\Local\Temp\ipykernel_11936\4088177958.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query, connection)


,age_band,demographic,total_sales,sales_share
0,unknown,unknown,1.606729e+10,0.4052
1,Retirees,Families,6.634687e+09,0.1673
2,Retirees,Couples,6.370580e+09,0.1607
3,Middle Aged,Families,4.354092e+09,0.1098
4,Young Adults,Couples,2.602923e+09,0.0656
5,Middle Aged,Couples,1.854160e+09,0.0468
6,Young Adults,Families,1.770889e+09,0.0447


# **9. ¿Podemos usar la columna avg_transaction para obtener el tamaño promedio de la transacción de cada año, tanto para Retail como para Shopify? Si no es posible, ¿cómo lo calcularías en su lugar?**

In [22]:
query = '''
    
    
SELECT
    calendar_year,
    platform,
    AVG(avg_transaction) as avg_avg_transactions,
    SUM(sales)/sum(transactions) as avg_transactions_group
FROM
    clean_weekly_sales

GROUP BY
    calendar_year, platform
ORDER BY 
    calendar_year
    

'''


pd.read_sql_query(query, connection)

C:\Users\kev_g\AppData\Local\Temp\ipykernel_11936\3787821212.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query, connection)


,calendar_year,platform,avg_avg_transactions,avg_transactions_group
0,2018,Retail,42.906369,36.5626
1,2018,Shopify,188.279272,192.4813
2,2019,Retail,41.968071,36.8335
3,2019,Shopify,177.559562,183.3611
4,2020,Retail,40.640231,36.5566
5,2020,Shopify,174.873569,179.0332


---

### 3. Análisis “Antes y Después”

Esta técnica se utiliza cuando queremos evaluar un evento importante y analizar su impacto **antes** y **después** de un punto específico en el tiempo.

Tomemos la fecha `week_date` del **15 de junio de 2020 (2020‑06‑15)** como la semana de referencia, cuando entraron en vigor los cambios de empaques sostenibles en Data Mart.

Incluiremos todos los valores de `week_date` **a partir del 15‑06‑2020** como el comienzo del período _después del cambio_, mientras que los valores de `week_date` **anteriores** pertenecerán al período _antes del cambio_.

Empleando este enfoque de análisis, responde a las siguientes preguntas:

# **1. ¿Cuál es el total de ventas en las 4 semanas anteriores y posteriores al 15‑06‑2020?**


In [23]:
query = '''
    
WITH params AS (
  SELECT 
    DATE '2020-06-15'         AS semana_base,
    4                        AS w                    -- nº de semanas antes/después
),


ventas_semanales AS (
  SELECT
    cw.week_date,
    SUM(cw.sales) AS total_sales
  FROM clean_weekly_sales  cw
  JOIN params p
    ON cw.week_date BETWEEN DATE_SUB(p.semana_base, INTERVAL p.w WEEK)
                        AND DATE_ADD(p.semana_base, INTERVAL p.w WEEK)
  GROUP BY cw.week_date
),


cambios_semanales AS (
  SELECT
    SUM(CASE WHEN week_date <  (SELECT semana_base FROM params)
             THEN total_sales END)                     AS before_week,

    SUM(CASE WHEN week_date >= (SELECT semana_base FROM params)
             THEN total_sales END)                     AS afer_week
  FROM ventas_semanales
)

SELECT
  afer_week - before_week                   AS sales_variance,
  ROUND(
       100 * (afer_week - before_week)
           / NULLIF(before_week, 0)
  , 2)                                                             AS variance_percentage
FROM cambios_semanales;

    

'''


pd.read_sql_query(query, connection)

C:\Users\kev_g\AppData\Local\Temp\ipykernel_11936\4174336333.py:45: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query, connection)


,sales_variance,variance_percentage
0,559052214.0,23.83


# **2. ¿Qué tal  las 12 semanas completas antes y después?**

In [24]:
query = '''
    
WITH params AS (
  SELECT 
    DATE '2020-06-15'         AS semana_base,
    12                        AS w                    -- nº de semanas antes/después
),


ventas_semanales AS (
  SELECT
    cw.week_date,
    SUM(cw.sales) AS total_sales
  FROM clean_weekly_sales  cw
  JOIN params p
    ON cw.week_date BETWEEN DATE_SUB(p.semana_base, INTERVAL p.w WEEK)
                        AND DATE_ADD(p.semana_base, INTERVAL p.w WEEK)
  GROUP BY cw.week_date
),


cambios_semanales AS (
  SELECT
    SUM(CASE WHEN week_date <  (SELECT semana_base FROM params)
             THEN total_sales END)                     AS before_week,

    SUM(CASE WHEN week_date >= (SELECT semana_base FROM params)
             THEN total_sales END)                     AS afer_week
  FROM ventas_semanales
)

SELECT
  afer_week - before_week                   AS sales_variance,
  ROUND(
       100 * (afer_week - before_week)
           / NULLIF(before_week, 0)
  , 2)                                                             AS variance_percentage
FROM cambios_semanales;

    

'''


pd.read_sql_query(query, connection)

C:\Users\kev_g\AppData\Local\Temp\ipykernel_11936\2661272245.py:45: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query, connection)


,sales_variance,variance_percentage
0,-152325394.0,-2.14
